In [1]:
import os
from langchain_chroma import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from pyprojroot import here
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
EMBEDDING_MODEL = GPT4AllEmbeddings()
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [3]:
vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=os.path.join(here(), "data/vectordb/swiss_airline_policy"),
    embedding_function=EMBEDDING_MODEL
)
print("Number of vectors in vectordb:",
    vectordb._collection.count(), "\n\n")

Number of vectors in vectordb: 22 




In [4]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"
docs = vectordb.similarity_search(message, k=K)
docs

[Document(id='096488c3-7b25-4938-bd48-9ede130cc505', metadata={'creationdate': '2024-09-08T21:34:59+00:00', 'creator': 'PDFCreator Online (www.pdfforge.org/online)', 'moddate': '2024-09-08T21:34:59+00:00', 'page': 9, 'page_label': '10', 'producer': 'PDFCreator Online (www.pdfforge.org/online)', 'source': 'data/unstructured_docs/swiss_airline_policy/swiss_faq.pdf', 'title': 'Merged with PDFCreator Online', 'total_pages': 12}, page_content="for a refund or may only be able to receive a partial refund. If you booked your flight through a third-party website or\ntravel agent, you may need to contact them directly to cancel your flight. Always check the terms and conditions of your\nticket to make sure you understand the cancellation policy and any associated fees or penalties. If you're cancelling your\nflight due to unforeseen circumstances such as a medical emergency or a natural disaster, Swiss Air may offer you\nspecial exemptions or accommodations. What is Swiss Airlines 24 Hour Cance

In [5]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

In [6]:
pprint(prompt)

('# Content:\n'
 'for a refund or may only be able to receive a partial refund. If you booked '
 'your flight through a third-party website or\n'
 'travel agent, you may need to contact them directly to cancel your flight. '
 'Always check the terms and conditions of your\n'
 'ticket to make sure you understand the cancellation policy and any '
 "associated fees or penalties. If you're cancelling your\n"
 'flight due to unforeseen circumstances such as a medical emergency or a '
 'natural disaster, Swiss Air may offer you\n'
 'special exemptions or accommodations. What is Swiss Airlines 24 Hour '
 'Cancellation Policy? Swiss Airlines has a 24\n'
 '\n'
 'How to Cancel a Swiss Air Flight: 877-\n'
 '5O7-7341 Step-by-Step Guide\n'
 'Swiss Air is a premium airline based in Switzerland that offers a range of '
 'domestic and international flights to\n'
 'passengers. However, sometimes situations arise where passengers may need to '
 'cancel their flights. In such cases, it is\n'
 'important 

In [7]:
llm = ChatGroq(model_name="deepseek-r1-distill-llama-70b")

In [ ]:
response = llm.invoke(
    input=[
        {"role": "system", "content": "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist. Only provide the answer"},
        {"role": "user", "content": prompt}
    ]
)

In [15]:
pprint(response.content[response.content.find("</think>")+len("</think>"):])

('\n'
 '\n'
 'The cancellation policy for Swiss Airlines depends on the fare type and '
 'timing:\n'
 '\n'
 '1. **Flexible Fares (Flex and Business Flex):** Allow cancellation up to 24 '
 'hours before departure without a penalty.\n'
 '2. **Non-Flexible Fares:** May incur fees, increasing as the departure date '
 'nears.\n'
 '3. **Changes:** Possible with fees depending on fare type.\n'
 '4. **Airline-Caused Cancellations:** May offer refunds or rebooking; '
 'exceptions apply for extraordinary circumstances.\n'
 '\n'
 'Check your fare type and contact Swiss Airlines early to avoid extra '
 'charges.')


In [16]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
        collection_name="rag-chroma",
        persist_directory=os.path.join(here(), "data/vectordb/swiss_airline_policy"),
        embedding_function=EMBEDDING_MODEL
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [17]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [18]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))

('for a refund or may only be able to receive a partial refund. If you booked '
 'your flight through a third-party website or\n'
 'travel agent, you may need to contact them directly to cancel your flight. '
 'Always check the terms and conditions of your\n'
 'ticket to make sure you understand the cancellation policy and any '
 "associated fees or penalties. If you're cancelling your\n"
 'flight due to unforeseen circumstances such as a medical emergency or a '
 'natural disaster, Swiss Air may offer you\n'
 'special exemptions or accommodations. What is Swiss Airlines 24 Hour '
 'Cancellation Policy? Swiss Airlines has a 24\n'
 '\n'
 'hour cancellation policy that allows passengers to cancel their flights '
 'within 24 hours of booking at +1-877-507-7341\n'
 'without penalty. This policy applies to all fare types, including '
 'non-refundable tickets. If you cancel your Swiss Airlines\n'
 "flight within 24 hours of booking, you'll receive a full refund of your "
 'ticket price.\n'
 

In [24]:
from glob import glob
files = glob("/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc/*")

In [25]:
files

['/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc/test.md',
 '/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc/info.md',
 '/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc/bangladesh.txt',
 '/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc/Sample-doc-file-1000kb.pdf',
 '/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc/Sample-doc-file-1000kb.doc']

In [28]:
os.path.join(os.path.split(files[0])[0],'info.md') in files

True

In [27]:
os.path.split(files[0])[0]

'/home/yato/ML/Agentic-Chatbot/data/unstructured_docs/test_doc'